In [ ]:
import numpy as np
import sep

In [ ]:
#additional setup
from astropy.io import fits
from astropy.visualization import make_lupton_rgb
from astropy.utils.data import get_pkg_data_filename
import matplotlib.pyplot as plt
from matplotlib import rcParams

%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]

In [ ]:

# read image into standard 2-d numpy array
filename = 'hubble125.fits'
data = fits.getdata(filename)


In [ ]:
# show the image

m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();
plt.savefig('HubbleImage.png',facecolor = 'white')

In [ ]:
# measure a spatially varying background on the image
data = data.byteswap().newbyteorder()
bkg = sep.Background(data)

In [ ]:
# get a "global" mean and noise of the image background:
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:

# evaluate background as 2-d array, same size as original image
bkg_image = bkg.back()
# bkg_image = np.array(bkg) # equivalent to above

In [ ]:
# show the background
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.savefig('Hubble Background.png',facecolor = 'white')
plt.colorbar()

In [ ]:

# evaluate the background noise as 2-d array, same size as original image
bkg_rms = bkg.rms()

In [ ]:

# show the background noise
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.savefig('HubbleBackgroundNoise.png',facecolor = 'white')
plt.colorbar();

In [ ]:
# subtract the background
data_sub = data - bkg


In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

In [ ]:
# how many objects were detected
length = len(objects)

In [ ]:
from matplotlib.patches import Ellipse

# plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

# plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
plt.savefig('HubbleObjects.png',facecolor = 'white')

In [ ]:
# available fields
objects.dtype.names

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

In [ ]:
# show the objects results:
for i in range(length):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

In [ ]:
bins = np.linspace(-.05,np.mean(flux),1000)

Histogram = plt.hist(flux,bins)

In [ ]:
mean = np.mean(flux)
median = np.median(flux)
SDeviation = np.std(flux)
outlier = max(flux)
outlierstd = outlier/SDeviation
location = np.where(flux == outlier)
x = float(objects['x'][location])
y = float(objects['y'][location])
print(f'The mean is {mean}')
print(f'The median is {median}')
print(f'Lastly the Standerd Deviation is {SDeviation}')
print(f'The outlier is {outlier} and it is {outlierstd} standerd deviations away')
print(f'It is located at {x} X,{y} Y')


Creating False Color Images